# 15 Ensemble Evaluation

Here I take the results from the hyperparameter search in step 14, create an ensemble, and evaluate the results.

In [133]:
import argparse
import csv
import itertools
import logging
import os
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd


In [134]:
dataX = pd.read_table('/Users/rah/Documents/omicsEye/btest/demo/X_dataset.txt', index_col=0, header=0)

In [135]:
dataY = pd.read_table('/Users/rah/Documents/omicsEye/btest/demo/Y_dataset.txt', index_col=0, header=0)

In [136]:
dataY

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S40,S41,S42,S43,S44,S45,S46,S47,S48,S49
0,4,5,2,3,1,3,4,5,6,4,...,3,5,6,4,1,4,4,6,1,2
1,4,5,3,2,1,4,3,5,6,4,...,2,5,6,2,1,4,3,6,2,2
2,4,5,3,2,1,4,3,5,6,4,...,3,5,6,3,1,4,3,6,1,1
3,4,5,2,3,1,4,3,5,6,4,...,3,5,6,3,1,4,3,6,1,2
4,4,5,2,2,1,4,3,5,6,4,...,3,5,6,3,1,4,3,6,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,4,5,6,1,6,1,3,2,4,5,...,1,3,1,3,1,2,3,3,4,6
496,4,5,6,3,6,3,1,1,2,5,...,2,1,4,2,3,4,1,1,4,6
497,1,3,6,4,5,4,1,2,3,6,...,2,3,4,3,1,4,1,3,5,5
498,3,4,6,2,6,2,4,1,5,5,...,2,4,1,3,3,2,4,1,4,6


In [137]:
ind = dataY.index.intersection(dataX.index)

In [138]:
dataX = dataY.loc[ind, :]
dataY = dataY.loc[ind, :]

In [139]:
df = pd.concat([dataX, dataY])
df = df.T
rho = df.corr()
pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)


In [140]:
rho

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,0.908986,0.934062,0.954532,0.927615,0.899869,0.886093,0.908986,0.916074,0.895895,...,-0.062405,-0.031874,-0.219311,-0.217679,-0.223753,-0.193384,-0.193384,-0.084059,-0.272342,-0.047617
1,0.908986,1.000000,0.957845,0.948121,0.945399,0.923679,0.897520,0.883991,0.933715,0.913335,...,0.053939,0.109229,-0.100074,-0.060128,-0.139566,-0.084571,-0.072350,0.013199,-0.212893,0.019310
2,0.934062,0.957845,1.000000,0.954863,0.946054,0.900527,0.898677,0.897520,0.958279,0.949751,...,0.004734,0.071501,-0.188219,-0.113743,-0.192228,-0.137892,-0.119781,-0.017146,-0.246564,0.000966
3,0.954532,0.948121,0.954863,1.000000,0.966679,0.908268,0.882415,0.911458,0.918639,0.916460,...,-0.017461,0.000984,-0.200308,-0.149706,-0.204746,-0.192515,-0.168053,-0.045744,-0.265900,-0.027397
4,0.927615,0.945399,0.946054,0.966679,1.000000,0.930238,0.904081,0.896849,0.928066,0.925679,...,-0.007450,0.035294,-0.171374,-0.138722,-0.199458,-0.144795,-0.181237,-0.065838,-0.284489,-0.035470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.193384,-0.084571,-0.137892,-0.192515,-0.144795,-0.094667,-0.059407,-0.145677,-0.143930,-0.079171,...,0.775664,0.757220,0.771065,0.724804,0.657534,1.000000,0.712573,0.584149,0.840998,0.694227
496,-0.193384,-0.072350,-0.119781,-0.168053,-0.181237,-0.149706,-0.143930,-0.176230,-0.125818,-0.043184,...,0.837147,0.775664,0.752965,0.761497,0.608611,0.712573,1.000000,0.700342,0.706458,0.785959
497,-0.084059,0.013199,-0.017146,-0.045744,-0.065838,-0.051859,-0.029221,-0.060128,-0.017146,0.034787,...,0.628106,0.597365,0.602130,0.565802,0.620841,0.584149,0.700342,1.000000,0.626957,0.724804
498,-0.272342,-0.212893,-0.246564,-0.265900,-0.284489,-0.290362,-0.198266,-0.249557,-0.270714,-0.241111,...,0.701885,0.646551,0.758998,0.626957,0.700342,0.840998,0.706458,0.626957,1.000000,0.718689


In [141]:
rho_long_format = rho.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'Correlation'})

In [142]:
rho_long_format

,Feature_1,Feature_2,Correlation
0,0,0,1.000000
1,0,1,0.908986
2,0,2,0.934062
3,0,3,0.954532
4,0,4,0.927615
...,...,...,...
999995,499,495,0.694227
999996,499,496,0.785959
999997,499,497,0.724804
999998,499,498,0.718689


In [143]:
pval_long_format = pval.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'P_value'})
pval_long_format

,Feature_1,Feature_2,P_value
0,0,0,0.000000e+00
1,0,1,7.186620e-20
2,0,2,4.188168e-23
3,0,3,7.053113e-27
4,0,4,3.650543e-22
...,...,...,...
999995,499,495,2.257120e-08
999996,499,496,1.365464e-11
999997,499,497,2.657403e-09
999998,499,498,4.168144e-09


In [144]:
result = pd.concat([rho_long_format, pval_long_format["P_value"]], axis=1)

In [145]:
result

,Feature_1,Feature_2,Correlation,P_value
0,0,0,1.000000,0.000000e+00
1,0,1,0.908986,7.186620e-20
2,0,2,0.934062,4.188168e-23
3,0,3,0.954532,7.053113e-27
4,0,4,0.927615,3.650543e-22
...,...,...,...,...
999995,499,495,0.694227,2.257120e-08
999996,499,496,0.785959,1.365464e-11
999997,499,497,0.724804,2.657403e-09
999998,499,498,0.718689,4.168144e-09


In [146]:
within_X = result[result["Feature_1"].isin(dataX.index) ]
within_X = within_X[within_X["Feature_2"].isin(dataX.index) ]


In [147]:
within_X

,Feature_1,Feature_2,Correlation,P_value
0,0,0,1.000000,0.000000e+00
1,0,1,0.908986,7.186620e-20
2,0,2,0.934062,4.188168e-23
3,0,3,0.954532,7.053113e-27
4,0,4,0.927615,3.650543e-22
...,...,...,...,...
999995,499,495,0.694227,2.257120e-08
999996,499,496,0.785959,1.365464e-11
999997,499,497,0.724804,2.657403e-09
999998,499,498,0.718689,4.168144e-09


In [148]:
within_Y = result[result["Feature_1"].isin(dataY.index)]
within_Y = within_Y[within_Y["Feature_2"].isin(dataY.index)]

In [149]:
X_Y = result[result["Feature_1"].isin(dataX.index) ]
X_Y = X_Y[X_Y["Feature_2"].isin(dataY.index) ]

In [152]:
def fdr(p_vals):
    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals, method='ordinal')
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1
    return fdr

In [153]:
def bh(p, q):
    from scipy.stats import rankdata
    pRank = rankdata(p, method='ordinal')
    m = len(p)
    p_adust = p *  len(p) / pRank
    max_i = 0
    for i in range(len(p)):
        if p_adust[i] <= q and max_i <= pRank[i]:
            max_i = pRank[i]
            p_threshold = p[i]
    return p_adust, p_threshold

In [154]:
fdr(X_Y["P_value"])

0         0.000000e+00
1         3.348064e-18
2         4.842373e-21
3         2.434627e-24
4         3.186300e-20
              ...     
999995    3.102144e-07
999996    2.332372e-10
999997    3.827787e-08
999998    5.933470e-08
999999    0.000000e+00
Name: P_value, Length: 1000000, dtype: float64

In [156]:
X_Y_p_adust, X_Y_p_threshold = bh(X_Y["P_value"], .1)
within_X_p_adust, within_X_p_threshold = bh(within_X["P_value"], .1)
within_Y_p_adust, within_Y_p_threshold = bh(within_Y["P_value"], .1)


In [158]:
X_Y_p_threshold


0.008015711735235301

In [159]:
within_X_p_threshold

0.008015711735235301

In [160]:
within_Y_p_threshold

0.008015711735235301

In [161]:
X_Y_p_adust
within_X_p_adust
within_Y_p_adust

0         0.000000e+00
1         3.348064e-18
2         4.842373e-21
3         2.434627e-24
4         3.186300e-20
              ...     
999995    3.102144e-07
999996    2.332372e-10
999997    3.827787e-08
999998    5.933470e-08
999999    0.000000e+00
Name: P_value, Length: 1000000, dtype: float64

In [162]:
X_Y["P_adusted"] = p_adust
within_X["P_adusted"] = within_X_p_adust
within_Y["P_adusted"] = within_Y_p_adust

In [163]:
X_Y
within_X
within_Y

,Feature_1,Feature_2,Correlation,P_value,P_adusted
0,0,0,1.000000,0.000000e+00,0.000000e+00
1,0,1,0.908986,7.186620e-20,3.348064e-18
2,0,2,0.934062,4.188168e-23,4.842373e-21
3,0,3,0.954532,7.053113e-27,2.434627e-24
4,0,4,0.927615,3.650543e-22,3.186300e-20
...,...,...,...,...,...
999995,499,495,0.694227,2.257120e-08,3.102144e-07
999996,499,496,0.785959,1.365464e-11,2.332372e-10
999997,499,497,0.724804,2.657403e-09,3.827787e-08
999998,499,498,0.718689,4.168144e-09,5.933470e-08


In [164]:
X_Y["bh_fdr_threshold"] = X_Y_p_threshold
within_X["bh_fdr_threshold"] = within_X_p_threshold
within_Y["bh_fdr_threshold"] = within_Y_p_threshold



In [165]:
X_Y

,Feature_1,Feature_2,Correlation,P_value,P_adusted,bh_fdr_threshold
0,0,0,1.000000,0.000000e+00,0.000000e+00,0.008016
1,0,1,0.908986,7.186620e-20,3.348064e-18,0.008016
2,0,2,0.934062,4.188168e-23,4.842373e-21,0.008016
3,0,3,0.954532,7.053113e-27,2.434627e-24,0.008016
4,0,4,0.927615,3.650543e-22,3.186300e-20,0.008016
...,...,...,...,...,...,...
999995,499,495,0.694227,2.257120e-08,3.102144e-07,0.008016
999996,499,496,0.785959,1.365464e-11,2.332372e-10,0.008016
999997,499,497,0.724804,2.657403e-09,3.827787e-08,0.008016
999998,499,498,0.718689,4.168144e-09,5.933470e-08,0.008016


In [166]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")